# Construction of promoter vector pYPKa_Z_RPL12A and terminator vector pYPKa_E_RPL12A

This notebook describe the construction of _E. coli_ vectors [pYPKa_Z_RPL12A](pYPKa_Z_RPL12A.gb) and [pYPKa_E_RPL12A](pYPKa_E_RPL12A.gb)
with the same insert for which PCR primers are also designed.

The insert defined below is cloned in pYPKa using the blunt restriction
enzymes [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html) and [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) in
two different plasmids. The insert cloned in [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html)
will be used as a promoter, while in the [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) site the insert will be used as a
terminator.

![pYPKa_Z and pYPKa_E plasmids](pYPK_ZE.png "pYPKa_Z and pYPKa_E plasmids")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.org/en/latest/) available online.
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
import pydna

The vector backbone [pYPKa](pYPKa.gb) is read from a local file.

In [2]:
pYPKa = pydna.read("pYPKa.gb")

Both restriction enzymes are imported from [Biopython](http://biopython.org)

In [3]:
from Bio.Restriction import ZraI, EcoRV

The vector is linearized with both enzymes.

In [4]:
pYPKa_ZraI  = pYPKa.linearize(ZraI)
pYPKa_EcoRV = pYPKa.linearize(EcoRV)

The insert sequence is read from a local file. This sequence was parsed from the ypkpathway data file.

In [5]:
ins = pydna.read("RPL12A.gb")

Primers for the terminator promoter need specific tails in order to produce
a [SmiI](http://rebase.neb.com/rebase/enz/SmiI.html) and a [PacI](http://rebase.neb.com/rebase/enz/PacI.html) 
when cloned in pYPKa in the EcoRV cloning position.

In [6]:
fp_tail = "ttaaat"
rp_tail = "taattaa"

Primers with the tails above are designed in the code cell below.

In [7]:
fp, rp = pydna.cloning_primers(ins, fp_tail=fp_tail, rp_tail=rp_tail, path="new_primers.txt")

The primers are included in the [new_primer.txt](new_primers.txt) list and in the end of the [pathway notebook](pw.ipynb) file.

In [8]:
print(fp.format("fasta"))

print(rp.format("fasta"))

>fw682 RPL12A
ttaaatTCTGGAATTTACCCTAACT

>rv682 RPL12A
taattaaTTTTGTATATCCTTTGTTCTT



PCR to create the insert using the newly designed primers.

In [9]:
prd = pydna.pcr(fp, rp, ins)

The PCR product has this length in bp.

In [10]:
len(prd)

695

A figure of the primers annealing on template.

In [11]:
prd.figure()

      5TCTGGAATTTACCCTAACT...AAGAACAAAGGATATACAAAA3
                             ||||||||||||||||||||| tm 42.7 (dbd) 53.9
                            3TTCTTGTTTCCTATATGTTTTaattaat5
5ttaaatTCTGGAATTTACCCTAACT3
       ||||||||||||||||||| tm 44.0 (dbd) 54.5
      3AGACCTTAAATGGGATTGA...TTCTTGTTTCCTATATGTTTT5

A suggested PCR program.

In [12]:
prd.program()


Taq (rate 30 nt/s) 35 cycles             |695bp
95.0°C    |95.0°C                 |      |SantaLucia 1998
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|
|         |      \ 52.0°C/ 0min20s| 5min |
|         |       \_____/         |      |
|         |         30s           |      |4-12°C

The final vectors are:

In [13]:
pYPKa_Z_RPL12A = (pYPKa_ZraI  + prd).looped().synced(pYPKa)
pYPKa_E_RPL12A = (pYPKa_EcoRV + prd).looped().synced(pYPKa)

The final vectors with reverse inserts are created below. These vectors theoretically make up
fifty percent of the clones. The PCR strategy below is used to identify the correct clones.

In [14]:
pYPKa_Z_RPL12Ab = (pYPKa_ZraI  + prd.rc()).looped().synced(pYPKa)
pYPKa_E_RPL12Ab = (pYPKa_EcoRV + prd.rc()).looped().synced(pYPKa)

A combination of standard primers and the newly designed primers are
used for the strategy to identify correct clones.
Standard primers are listed [here](standard_primers.txt).

In [15]:
p = { x.id: x for x in pydna.parse("standard_primers.txt") }

## Diagnostic PCR confirmation

The correct structure of pYPKa_Z_RPL12A is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the RPL12Afw primer specific for the insert
in a multiplex PCR reaction with
all three primers present.

Two PCR products are expected if the insert was cloned, the sizes depend
on the orientation. If the vector is empty or contains another insert, only one
product is formed.

#### Expected PCR products sizes from pYPKa_Z_RPL12A:

pYPKa_Z_RPL12A with insert in correct orientation.

In [16]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_Z_RPL12A).products

[Dseqrecord(-1629), Dseqrecord(-1461)]

pYPKa_Z_RPL12A with insert in reverse orientation.

In [17]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_Z_RPL12Ab).products

[Dseqrecord(-1629), Dseqrecord(-863)]

Empty pYPKa clone.

pydna.Anneal( (p['577'], p['342'], fp), pYPKa).products

#### Expected PCR products sizes pYPKa_E_RPL12A:

pYPKa_E_RPL12A with insert in correct orientation.

In [18]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_E_RPL12A).products

[Dseqrecord(-1629), Dseqrecord(-1380)]

pYPKa_E_RPL12A with insert in reverse orientation.

In [19]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_E_RPL12Ab).products


[Dseqrecord(-1629), Dseqrecord(-944)]

The cseguid checksum for the resulting plasmids are calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid)
uniquely identifies a circular double stranded sequence.

In [20]:
print(pYPKa_Z_RPL12A.cseguid())
print(pYPKa_E_RPL12A.cseguid())

5V7tFXwcW0MoRk9uKZfrIxaDNsk
gwiaBn3DcsP_k2VHcfsjrZMkyro


The sequences are named based on the name of the cloned insert.

In [21]:
pYPKa_Z_RPL12A.locus = "pYPKa_Z_RPL12A"[:16]
pYPKa_E_RPL12A.locus = "pYPKa_Z_RPL12A"[:16]

Sequences are stamped with the cseguid checksum.
This can be used to verify the integrity of the sequence file.

In [22]:
pYPKa_Z_RPL12A.stamp()
pYPKa_E_RPL12A.stamp()

cSEGUID_gwiaBn3DcsP_k2VHcfsjrZMkyro_2015-10-02T06:18:24.232856

Sequences are written to local files.

In [23]:
pYPKa_Z_RPL12A.write("pYPKa_Z_RPL12A.gb")
pYPKa_E_RPL12A.write("pYPKa_E_RPL12A.gb")

###[pYPKa_E_RPL12A](pYPKa_E_RPL12A.gb)

# Download [pYPKa_Z_RPL12A](pYPKa_Z_RPL12A.gb)

In [24]:
import pydna
reloaded = pydna.read("pYPKa_Z_RPL12A.gb")
reloaded.verify_stamp()

cSEGUID_5V7tFXwcW0MoRk9uKZfrIxaDNsk

# Download [pYPKa_E_RPL12A](pYPKa_E_RPL12A.gb)

In [25]:
import pydna
reloaded = pydna.read("pYPKa_E_RPL12A.gb")
reloaded.verify_stamp()

cSEGUID_gwiaBn3DcsP_k2VHcfsjrZMkyro